In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec
from sklearn import preprocessing
from tqdm import tqdm
import multiprocessing
warnings.filterwarnings('ignore')
print(multiprocessing.cpu_count())
import gc
from category_encoders import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
## You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
origin_features = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

4
/kaggle/input/financial-risk/test_a_trans.csv
/kaggle/input/financial-risk/test_a_base.csv
/kaggle/input/financial-risk/feature_importance_df 32_after.csv
/kaggle/input/financial-risk/submit_example.csv
/kaggle/input/financial-risk/train_label.csv
/kaggle/input/financial-risk/feature_importance_df(32ming) (1).csv
/kaggle/input/financial-risk/train_trans.csv
/kaggle/input/financial-risk/train_base.csv
/kaggle/input/financial-risk/.doc
/kaggle/input/financial-risk/all_precess_data.csv
/kaggle/input/financial-risk/test_a_op.csv
/kaggle/input/financial-risk/train_op.csv
/kaggle/input/financial-risk/test_a_trans.csv
/kaggle/input/financial-risk/test_a_base.csv
/kaggle/input/financial-risk/feature_importance_df 32_after.csv
/kaggle/input/financial-risk/submit_example.csv
/kaggle/input/financial-risk/train_label.csv
/kaggle/input/financial-risk/feature_importance_df(32ming) (1).csv
/kaggle/input/financial-risk/train_trans.csv
/kaggle/input/financial-risk/train_base.csv
/kaggle/input/financi

In [2]:
%%time
def gen_user_status_features(df,value):
    group_df = df.groupby(['user'])[value].agg(
        a='mean',
        b= 'std',
        c='max',
        d='min',
        e='sum',
        f='median',
        g='count',
        h='median',
        i='skew',
        ).reset_index()
    group_df['j']=group_df['b'] / group_df['a']
    group_df.rename(columns={
                    'a':'user_{}_mea'.format(value),
                    'b':'user_{}_std'.format(value),
                    'c':'user_{}_max'.format(value),
                    'd':'user_{}_min'.format(value),
                    'e':'user_{}_sum'.format(value),
                    'f':'user_{}_med'.format(value),
                    'g':'user_{}_cnt'.format(value),
                    'h':'user_{}_median'.format(value),
                    'i':'user_{}_skew'.format(value),
                    'j':'user_{}_CV'.format(value)
                    },
                   inplace=True
                   )
    return group_df
def gen_city_status_features(df,value):
    group_df = df.groupby(['city'])[value].agg(
        a='mean',
        b= 'std',
        c='max',
        d='min',
        e='sum',
        f='median',
        g='count',
        h='median',
        i='skew',
        ).reset_index()
    group_df['j']=group_df['b'] / group_df['a']
    group_df.rename(columns={
                    'a':'city_{}_mea'.format(value),
                    'b':'city_{}_std'.format(value),
                    'c':'city_{}_max'.format(value),
                    'd':'city_{}_min'.format(value),
                    'e':'city_{}_sum'.format(value),
                    'f':'city_{}_med'.format(value),
                    'g':'city_{}_cnt'.format(value),
                    'h':'city_{}_median'.format(value),
                    'i':'city_{}_skew'.format(value),
                    'j':'city_{}_CV'.format(value)
                    },
                   inplace=True
                   )
    return group_df
    
def gen_user_group_features(df, col,value):
    
    group_df = df.pivot_table(index='user',
                              columns=col,
                              values=value,
                              dropna=False,
                              aggfunc=['count', 'sum','nunique','mean','std','max','min']).fillna(0)
    cols=group_df.columns
    group_df.columns = ['user_group_{}_{}_{}_{}'.format(col,value, f[1], f[0]) for f in group_df.columns]
    for f in cols:
        group_df['user_group_{}_{}_{}_CV'.format(col,value,f[1])] = group_df['user_group_{}_{}_{}_std'.format(col,value,f[1])] / group_df['user_group_{}_{}_{}_mean'.format(col,value,f[1])]
    group_df.reset_index(inplace=True)
    return group_df

def gen_user_group_nunique_features(df, col,value):
    
    group_df = df.pivot_table(index='user',
                              columns=col,
                              values=value,
                              dropna=False,
                              aggfunc=['count','nunique']).fillna(0)
    group_df.columns = ['user_{}_{}_{}_{}'.format(col,value, f[1], f[0]) for f in group_df.columns]
    group_df.reset_index(inplace=True)
    return group_df

def gen_user_nunique_features(df, value, prefix):
    a="user_{}_{}_nuniq".format(prefix, value)
    b="user_{}_{}_count".format(prefix, value)
    group_df = df.groupby(['user'])[value].agg(
        a='nunique',b='count'
    ).reset_index()
    
    group_df.rename(columns={'a':a,'b':b},inplace=True) 
    return group_df

def gen_user_null_features(df, value, prefix):
    df['is_null'] = 0
    df.loc[df[value].isnull(), 'is_null'] = 1

    group_df = df.groupby(['user'])['is_null'].agg(a='sum',
                                                    b='mean').reset_index()
    
    group_df.rename(columns={'a':'user_{}_{}_null_cnt'.format(prefix, value),
                    'b':'user_{}_{}_null_ratio'.format(prefix, value)},
                   inplace=True
                   )

    return group_df

    
def gen_user_window_amount_features(df, window):
    
    group_df = df[df['days_diff']>window].groupby('user')['amount'].agg(
        a='mean',
         b='std',
         c='max',
         d='min',
         e='sum',
        f='median',
         g='count',
        ).reset_index()
    
    group_df.rename(columns={'a':'user_amount_mean_{}d'.format(window),
                    'b':'user_amount_std_{}d'.format(window),
                    'c':'user_amount_max_{}d'.format(window),
                    'd':'user_amount_min_{}d'.format(window),
                    'e':'user_amount_sum_{}d'.format(window),
                    'f':'user_amount_med_{}d'.format(window),
                    'g':'user_amount_cnt_{}d'.format(window)},
                   inplace=True
                   )
    
    return group_df

def gen_user_window_hour_amount_features(df, window):
    
    group_df = df[df['hour']>window].groupby('user')['amount'].agg(
        a='mean',
         b='std',
         c='max',
         d='min',
         e='sum',
         f='median',
         g='count',
        ).reset_index()
    
    group_df.rename(columns={'a':'user_amount_mean_{}h'.format(window),
                    'b':'user_amount_std_{}h'.format(window),
                    'c':'user_amount_max_{}h'.format(window),
                    'd':'user_amount_min_{}h'.format(window),
                    'e':'user_amount_sum_{}h'.format(window),
                    'f':'user_amount_med_{}h'.format(window),
                    'g':'user_amount_cnt_{}h'.format(window)},
                   inplace=True
                   )
    
    return group_df


def gen_user_tfidf_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = TfidfVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=5, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_tfidf_{}_{}'.format(value, i) for i in range(5)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

def gen_user_countvec_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = CountVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=5, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_countvec_{}_{}'.format(value, i) for i in range(5)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

#定义加载函数
def load_dataset(DATA_PATH):
    train_label = pd.read_csv(DATA_PATH+'train_label.csv')
    test_base = pd.read_csv(DATA_PATH+'test_a_base.csv')
    
    train_base = pd.read_csv(DATA_PATH+'train_base.csv')
    train_op = pd.read_csv(DATA_PATH+'train_op.csv')
    train_trans = pd.read_csv(DATA_PATH+'train_trans.csv')
    test_op = pd.read_csv(DATA_PATH+'test_a_op.csv')
    test_trans = pd.read_csv(DATA_PATH+'test_a_trans.csv')
    origin_features = list(train_base.columns)+list(train_trans.columns)+list(train_op.columns)
    return train_label, train_base, test_base, train_op, train_trans, test_op, test_trans

#tran_trans和train_op文件中的tm_diff时间转换
def transform_time(x):
    day = int(x.split(' ')[0])
    hour = int(x.split(' ')[2].split('.')[0].split(':')[0])
    minute = int(x.split(' ')[2].split('.')[0].split(':')[1])
    second = int(x.split(' ')[2].split('.')[0].split(':')[2])
    return 86400*day+3600*hour+60*minute+second



#讲所有的数据进行初步的整合，并且把时间days_diff进行了数字化提取
def data_preprocess(DATA_PATH):
    train_label, train_base, test_base, train_op, train_trans, test_op, test_trans = load_dataset(DATA_PATH=DATA_PATH)
    # 拼接数据
    train_df = train_base.copy()
    test_df = test_base.copy()
        #将train_base数据和train_label整合
    train_df = train_label.merge(train_df, on=['user'], how='left')
    del train_base, test_base
        #将train_op数据和test_op整合--方便将train和test中所有的数据进行统一的处理
    op_df = pd.concat([train_op, test_op], axis=0, ignore_index=True)
        #将train_trans数据和test_trans整合--方便将train和test中所有的数据进行统一的处理
    trans_df = pd.concat([train_trans, test_trans], axis=0, ignore_index=True)
        #将train_base数据和test_base整合
    data = pd.concat([train_df, test_df], axis=0, ignore_index=True) 
    del train_op, test_op, train_df, test_df
    
    
    # 时间维度的处理
    op_df['days_diff'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
    trans_df['days_diff'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
    
    op_df['timestamp'] = op_df['tm_diff'].apply(lambda x: transform_time(x))
    trans_df['timestamp'] = trans_df['tm_diff'].apply(lambda x: transform_time(x))
    op_df['hour'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
    trans_df['hour'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
    trans_df['week'] = trans_df['days_diff'].apply(lambda x: x % 7)
    
    
    # 排序
    trans_df = trans_df.sort_values(by=['user', 'timestamp'])
    op_df = op_df.sort_values(by=['user', 'timestamp'])
    trans_df.reset_index(inplace=True, drop=True)
    op_df.reset_index(inplace=True, drop=True)

    gc.collect()
    return data, op_df, trans_df


def kfold_stats_feature(train, test, feats, k):
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)  # 这里最好和后面模型的K折交叉验证保持一致

    train['fold'] = None
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['label'])):
        train.loc[val_idx, 'fold'] = fold_

    kfold_features = []
    for feat in feats:
        nums_columns = ['label']
        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            kfold_features.append(colname)
            train[colname] = None
            for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['label'])):
                tmp_trn = train.iloc[trn_idx]
                order_label = tmp_trn.groupby([feat])[f].mean()
                tmp = train.loc[train.fold == fold_, [feat]]
                train.loc[train.fold == fold_, colname] = tmp[feat].map(order_label)
                # fillna
                global_mean = train[f].mean()
                train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, colname].fillna(global_mean)
            train[colname] = train[colname].astype(float)

        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            test[colname] = None
            order_label = train.groupby([feat])[f].mean()
            test[colname] = test[feat].map(order_label)
            # fillna
            global_mean = train[f].mean()
            test[colname] = test[colname].fillna(global_mean)
            test[colname] = test[colname].astype(float)
    del train['fold']
    return train, test
def w2v_feat(df, feat, mode):
    data_frame=df.copy()
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True)

    print(f'Start {mode} word2vec ...')
    model = Word2Vec(data_frame[feat].values.tolist(), size=10, window=2, min_count=1,
                     workers=multiprocessing.cpu_count(), iter=10)
    stat_list = ['min', 'max', 'mean', 'std']
    new_all = pd.DataFrame()
    for m, t in enumerate(feat):
        print(f'Start gen feat of {t} ...')
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)
        w2c_list = [f'w2c_{t}_{n}' for n in range(10)]
        tmp_df.columns = [t] + w2c_list
        tmp_df = data_frame[['user', t]].merge(tmp_df, on=t)
        tmp_df = tmp_df.drop_duplicates().groupby('user').agg(stat_list).reset_index()
        tmp_df.columns = ['user'] + [f'{p}_{q}' for p in w2c_list for q in stat_list]
        if m == 0:
            new_all = pd.concat([new_all, tmp_df], axis=1)
        else:
            new_all = pd.merge(new_all, tmp_df, how='left', on='user')
    return new_all

def gen_features(df, op, trans):
    
    # base数据处理
    print("base logistic features")
    df['card_a_cnt_avg'] = df["card_a_cnt"]/df["using_time"]
    df['product7_fail_ratio'] = df['product7_fail_cnt'] / df['product7_cnt']
    df['city_count'] = df.groupby(['city'])['user'].transform('count')  ## 与原始数据集相同数量的项目
    df['city_product7_fail_ratio'] =  df.groupby(['city'])['product7_fail_cnt'].transform('sum')/df.groupby(['city'])['product7_cnt'].transform('sum')
    df['province_count'] = df.groupby(['province'])['user'].transform('count')
    df['province_product7_fail_ratio'] = df.groupby(['province'])['product7_fail_cnt'].transform('sum')/df.groupby(['province'])['product7_cnt'].transform('sum')
    
    df['login_cnt_period_var']=df[['login_cnt_period1','login_cnt_period2']].std(axis=1)
    df['login_cnt_period_CV']=df['login_cnt_period_var'] / df[['login_cnt_period1','login_cnt_period2']].mean(axis=1)
    df['card_var']=df[['card_a_cnt','card_b_cnt','card_c_cnt','card_d_cnt']].std(axis=1)
    df['card_CV']=df['card_var'] / df[['card_a_cnt','card_b_cnt','card_c_cnt','card_d_cnt']].mean(axis=1)
    df['op_cnt_var']=df[['op1_cnt','op2_cnt']].std(axis=1)
    df['op_cnt_CV']=df['op_cnt_var'] / df[['op1_cnt','op2_cnt']].mean(axis=1)
    df['service_var']=df[['service1_cnt','service2_cnt']].std(axis=1)
    df['service_CV']=df['service_var'] / df[['service1_cnt','service2_cnt']].mean(axis=1)
    df['login_var']=df[['login_cnt_period1','login_cnt_period2','login_cnt_avg']].std(axis=1)
    df['login_CV']=df['login_var'] / df[['login_cnt_period1','login_cnt_period2','login_cnt_avg']].mean(axis=1)
    
    
    df['ip_cnt_per_login_cnt_avg'] = df['ip_cnt']/df[['login_cnt_period1','login_cnt_period2','login_cnt_avg']].mean(axis=1)
    df['ip_cnt_per_login_days_cnt'] = df['ip_cnt']/df['login_days_cnt']
    df['acc_count_per_time'] = df["acc_count"]/df["using_time"]
    df['ip_cnt_per_using_time'] = df["ip_cnt"]/df["using_time"]
    df['agreement_total_per_using_time'] = df["agreement_total"]/df["using_time"]
    df['agreement_total_per_account_cnt'] = df["agreement_total"]/df["acc_count"]
    df['ip_cnt_per_acc_count'] = df["ip_cnt"]/df["acc_count"]
    df['service1_amt_per_using_time'] = df["service1_amt"]/df["using_time"]
    df["service_cnt_per_time"]=(df["service1_cnt"]+df["service1_cnt"])/df["using_time"]
    df["service_cnt_per_time"]=(df["service1_cnt"]+df["service1_cnt"])/df["using_time"]
    
    df['op_cnt_per_usign_time'] = (df["op1_cnt"]+df["op2_cnt"])/df["using_time"]
    df['op_cnt_per_login_cnt_avg'] = (df["op1_cnt"]+df["op2_cnt"])/df['login_cnt_avg']
    df['op_cnt_per_login_cnt_p1'] =  (df["op1_cnt"]+df["op2_cnt"])/df['login_cnt_period1']
    df['op_cnt_per_login_cnt_p2'] =  (df["op1_cnt"]+df["op2_cnt"])/df['login_cnt_period2']

    # trans
    df = df.merge(gen_user_status_features(trans,'amount'), on=['user'], how='left')
    df = df.merge(gen_user_status_features(trans,'days_diff'), on=['user'], how='left')
    df = df.merge(gen_user_status_features(trans,'hour'), on=['user'], how='left')
    df=df.merge(df.groupby(['city'])['user_amount_sum'].agg(city_amount_mean='mean',city_amount_sum='sum',city_amount_std='std').reset_index(),
             on=['city'],how='left')
  
    print("trans nunique && null features")
    for col in tqdm(['days_diff', 'platform', 'tunnel_in', 'tunnel_out', 'type1', 'type2','ip', 'ip_3']):
        df = df.merge(gen_user_nunique_features(df=trans, value=col, prefix='trans'), on=['user'], how='left')
        df = df.merge(gen_user_null_features(df=trans, value=col, prefix='trans'), on=['user'], how='left')
    df['user_amount_per_days'] = df['user_amount_sum'].div(df['user_trans_days_diff_nuniq']) 
    df['user_amount_per_cnt'] = df['user_amount_sum'] / df['user_amount_cnt']
    df['user_amount_per_tunnel_in'] = df['user_amount_sum'] / df['user_trans_tunnel_in_nuniq']
    df['user_amount_per_cnt'] = df['user_amount_sum'] / df['user_trans_tunnel_out_nuniq']
    df['user_amount_per_platform'] = df['user_amount_sum'] / df['user_trans_platform_nuniq']
    df=df.merge(gen_city_status_features(df,'user_trans_ip_null_cnt'),on=['city'],how='left')
    df=df.merge(gen_city_status_features(df,'user_trans_ip_count'),on=['city'],how='left')
    df['city_ipcnt_vs_nullipcnt']=df['city_user_trans_ip_count_sum'] / df['city_user_trans_ip_null_cnt_sum']
    
    
    temp=trans[trans['ip'].isnull()].groupby(['user'])['amount'].agg(user_null_ip_amount_sum='sum').reset_index()
    df=df.merge(temp,on=['user'],how='left')
    df['user_null_ip_amount_sum'].fillna(0,inplace=True)
    df['user_null_ip_amount_percent'] = df['user_null_ip_amount_sum'] / df['user_amount_sum']
    df['user_ip_amount_percent'] = 1-df['user_null_ip_amount_percent'] 
    df=df.merge(gen_city_status_features(df,'user_null_ip_amount_sum'),on=['city'],how='left')
    df['city_null_ip_amount_percent'] = df['city_user_null_ip_amount_sum_sum'] / df['city_amount_sum']
    
    
    print("trans group amount features")
    trans_type1=trans.query("type1==['f67d4b5a05a1352a','19d44f1a51919482','674e8d5860bc033d', '45a1168437c708ff']")
    trans_type2=trans.query("type2==['11a213398ee0c623']")
    trans_plat=trans.query("platform==['46c69cbbce5f1568','42573d7287a8c9c2']")
    trans_tunnel_in=trans.query("tunnel_in==['b2e7fa260df4998d','9c5701005a2d85bd']")
    df = df.merge(gen_user_group_features(df=trans_type2,col='type2',      value='amount'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_type1,col='type1',      value='amount'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_tunnel_in,col='tunnel_in',  value='amount'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans,col='tunnel_out',value='amount'), on=['user'], how='left')
    df = df.merge(gen_user_group_features(df=trans_plat,col='platform',   value='amount'),on=['user'],how='left')
    
    print("trans group days_diff features")
    df = df.merge(gen_user_group_features(df=trans_type2,col='type2',      value='days_diff'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_type1,col='type1',      value='days_diff'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_tunnel_in,col='tunnel_in',  value='days_diff'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans,col='tunnel_out',value='days_diff'), on=['user'], how='left')
    df = df.merge(gen_user_group_features(df=trans_plat,col='platform',   value='days_diff'),on=['user'],how='left')
    
    
    print("trans group nunique features")
    df = df.merge(gen_user_group_nunique_features(df=trans,col='platform',   value='type1'),on=['user'],how='left')
    df = df.merge(gen_user_group_nunique_features(df=trans,col='platform',   value='ip'),on=['user'],how='left')
    
    print("trans days window features")
    for window in tqdm([20,13,15]):
        df = df.merge(gen_user_window_amount_features(df=trans, window=window), on=['user'], how='left')
        
    print("trans hour window features")
    for win_hour in tqdm([9,18,19,20]):
        df = df.merge(gen_user_window_hour_amount_features(df=trans, window=win_hour), on=['user'], how='left')
        
    print("trans tfidf--count features")    
    df = df.merge(gen_user_tfidf_features(df=trans, value='type1'), on=['user'], how='left')
    df = df.merge(gen_user_tfidf_features(df=trans, value='type2'), on=['user'], how='left')
    
    # op
    print("op nuique features")
    for col in tqdm(['op_type','op_mode','channel']):
        df = df.merge(gen_user_nunique_features(df=op, value=col, prefix='op'), on=['user'], how='left')
    
    print("op null features")
    for col in tqdm(['op_device', 'ip', 'net_type', 'channel', 'ip_3']):
        df = df.merge(gen_user_null_features(df=op, value=col, prefix='op'), on=['user'], how='left')
    
    print("op days_diff features")
    df = df.merge(gen_user_status_features(op,'days_diff'), on=['user'], how='left')
    df = df.merge(gen_user_status_features(op,'hour'), on=['user'], how='left')

    print("op tfidf--cnt features")
    df = df.merge(gen_user_tfidf_features(df=op, value='op_mode'), on=['user'], how='left')
    df = df.merge(gen_user_tfidf_features(df=op, value='op_type'), on=['user'], how='left')
    df = df.merge(gen_user_tfidf_features(df=op, value='op_device'), on=['user'], how='left')
    df = df.merge(gen_user_countvec_features(df=op, value='op_mode'), on=['user'], how='left')
    df = df.merge(gen_user_countvec_features(df=op, value='op_type'), on=['user'], how='left')
    df = df.merge(gen_user_countvec_features(df=op, value='op_device'), on=['user'], how='left')
    
    df=df.merge(w2v_feat(op,["op_device","ip","channel","net_type","ip_3"],"train"),on=['user'],how='left')

    return df


def lgb_model_origin(train, target, test, k):
    feats = [f for f in train.columns if f not in ['user', 'label']]
    print('Current num of features:', len(feats))
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)
    oof_probs = np.zeros(train.shape[0])
    output_preds = 0
    offline_score = []
    valid_score = []
    train_score = []
    feature_importance_df = pd.DataFrame()
    #parameters = {
    #    'learning_rate': 0.05,
    #    'boosting_type': 'gbdt',
    #    'objective': 'binary',
    #    'metric': 'auc',
    #    'num_leaves': 63,
    #    'feature_fraction': 0.8,
    #    'bagging_fraction': 0.8,
    #    'min_data_in_leaf': 20,
    #    'verbose': -1,
    #    'nthread': 8
    #}
    parameters = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 50,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'min_data_in_leaf': 20,
        'reg_alpha':10,
        'reg_lambda':8,
        'verbose': -1,
        'nthread': 8,
        'colsample_bytree':0.77,
        'min_child_weight':4,
        'min_child_samples':10,
        'min_split_gain':0,
        'lambda_l1': 0.8,
    }

    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
        dtrain = lgb.Dataset(train_X,
                             label=train_y)
        dval = lgb.Dataset(test_X,
                           label=test_y)
        lgb_model = lgb.train(
                parameters,
                dtrain,
                num_boost_round=5000,
                valid_sets=[dval],
                early_stopping_rounds=100,
                verbose_eval=100,
        )
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        output_preds += lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration)/folds.n_splits
        
        offline_score.append(lgb_model.best_score['valid_0']['auc'])
        print(offline_score)
        
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    valid_auc = roc_auc_score(target, oof_probs)
    print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print("valid_auc:  {}".format(valid_auc))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(40))

    return output_preds, oof_probs, np.mean(offline_score),feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False)
# This way we have randomness and are able to reproduce the behaviour within this cell.
          

CPU times: user 14 µs, sys: 3 µs, total: 17 µs
Wall time: 25.7 µs


In [3]:

def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encoding(train,test, feature, k,sigma=1.0,a=1.0):
    
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)    
    
    print("Target encoding")
    for col in tqdm(feature):
        #train['target_'+col] = 0
        #test['target_' + col] =0
        train['catboost_'+col] = 0
        test['catboost_' + col] =0
        if col not in origin_features:
            #smoothing=50
            #noise_level=0.7
            sigma=1.5
            a=5.0
        for trn_idx,val_idx in kf.split(train,train['label']):
            X = train.iloc[trn_idx][col]
            #enc_tar = TargetEncoder(smoothing=smoothing).fit(X,train.iloc[trn_idx]['label'])
            enc_cat = CatBoostEncoder(sigma=sigma).fit(X,train.iloc[trn_idx]['label'])
            #train['target_'+col] += add_noise(enc_tar.transform(train[col])[col],noise_level) / k
            #test['target_'+col] += add_noise(enc_tar.transform(test[col])[col],noise_level) / k
            train['catboost_'+col] += enc_cat.transform(train[col])[col] / k
            test['catboost_'+col] += enc_cat.transform(test[col])[col] / k
    print('Target encoding finish')
    return  train,test
            

In [4]:
%%time
DATA_PATH = '/kaggle/input/financial-risk/'
print('读取数据...')
base_df, op_df, trans_df = data_preprocess(DATA_PATH=DATA_PATH)


读取数据...
CPU times: user 48.7 s, sys: 4.95 s, total: 53.7 s
Wall time: 54.2 s


In [5]:
%%time
print('开始特征工程...')
data = gen_features(base_df, op_df, trans_df)
data.to_csv('data.csv')

开始特征工程...
base logistic features


  0%|          | 0/8 [00:00<?, ?it/s]

trans nunique && null features


100%|██████████| 8/8 [00:07<00:00,  1.00it/s]


trans group amount features
trans group days_diff features
trans group nunique features


...

op nuique features


  0%|          | 0/5 [00:00<?, ?it/s]

op null features


100%|██████████| 5/5 [00:12<00:00,  2.47s/it]


op days_diff features
op tfidf--cnt features
Start train word2vec ...
Start gen feat of op_device ...
Start gen feat of ip ...
Start gen feat of channel ...
Start gen feat of net_type ...
Start gen feat of ip_3 ...
CPU times: user 18min 19s, sys: 20.1 s, total: 18min 39s
Wall time: 11min 43s


In [6]:
#data['city_level'] = data['city'].map(str) + '_' + data['level'].map(str)
#data['city_product1_amount'] = data['city'].map(str) + '_' + data['product1_amount'].map(str)
#data['city_balance1_avg'] = data['city'].map(str) + '_' + data['balance1_avg'].map(str)
#data['city_balance2_avg'] = data['city'].map(str) + '_' + data['balance2_avg'].map(str)
#data['city_balance_avg'] = data['city'].map(str) + '_' + data['balance_avg'].map(str)
target_encode_cols = [
'user_trans_ip_count',   
'product7_fail_ratio',
'product7_fail_cnt',
'user_amount_med_9h',
'service3',
'age',
'login_cnt_period_CV',
'product7_cnt',
'svd_countvec_op_type_3',
'svd_countvec_op_mode_3',
'svd_tfidf_op_type_3',
'using_time'
]


In [7]:
%%time
print('开始模型训练...')
train = data[~data['label'].isnull()].copy()
target = train['label']
test = data[data['label'].isnull()].copy()

#train, test = kfold_stats_feature(train, test, target_encode_cols, 8)
train,test = target_encoding(train,test,target_encode_cols,k=8)

data = pd.concat([train, test], axis=0, ignore_index=True) 
for col in tqdm([f for f in data.select_dtypes('object').columns if f not in ['user']]):
    le = LabelEncoder()
    data[col].fillna("-1", inplace=True)
    data[col] = le.fit_transform(data[col])
    
train = data[~data['label'].isnull()].copy()
test = data[data['label'].isnull()].copy()
#train.loc[train.fold == fold_, colname]=train.iloc[trn_idx].groupby([feat])[f].transform('mean')
train.drop(['service3_level'], axis=1, inplace=True)
test.drop(['service3_level'], axis=1, inplace=True)
#train.drop(target_encode_cols, axis=1, inplace=True)
#test.drop(target_encode_cols, axis=1, inplace=True)

lgb_preds, lgb_oof, lgb_score,feature_importance_df = lgb_model_origin(train=train, target=target, test=test, k=8)

sub_df = test[['user']].copy()
sub_df['prob'] = lgb_preds
sub_df.to_csv('sub(8folds).csv', index=False)
feature_importance_df.to_csv('feature(1).csv')

开始模型训练...


  0%|          | 0/12 [00:00<?, ?it/s]

Target encoding


100%|██████████| 12/12 [00:51<00:00,  4.31s/it]


Target encoding finish


100%|██████████| 25/25 [00:01<00:00, 17.47it/s]


Current num of features: 746
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.737254
[200]	valid_0's auc: 0.738362
[300]	valid_0's auc: 0.738113
Early stopping, best iteration is:
[218]	valid_0's auc: 0.738675
[0.7386746182007863]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.74499
[200]	valid_0's auc: 0.745613
Early stopping, best iteration is:
[130]	valid_0's auc: 0.748426
[0.7386746182007863, 0.7484257725336398]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.731833
[200]	valid_0's auc: 0.732699
Early stopping, best iteration is:
[162]	valid_0's auc: 0.733984
[0.7386746182007863, 0.7484257725336398, 0.7339841593120615]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.725435
[200]	valid_0's auc: 0.727196
[300]	valid_0's auc: 0.728543
Early stopping, best iteration is:
[268]	valid_0's auc: 0.72877
[0.7386746182007863, 0.7484257725336398, 0

In [8]:
%%time
print('开始模型训练...')
lgb_preds, lgb_oof, lgb_score,feature_importance_df = lgb_model_origin(train=train, target=target, test=test, k=10)

sub_df = test[['user']].copy()
sub_df['prob'] = lgb_preds
sub_df.to_csv('sub(10folds).csv', index=False)
feature_importance_df.to_csv('feature(2).csv')

开始模型训练...
Current num of features: 746
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.741433
[200]	valid_0's auc: 0.74257
[300]	valid_0's auc: 0.74352
[400]	valid_0's auc: 0.743421
[500]	valid_0's auc: 0.743912
Early stopping, best iteration is:
[498]	valid_0's auc: 0.744054
[0.7440540710017827]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.746102
[200]	valid_0's auc: 0.747838
Early stopping, best iteration is:
[135]	valid_0's auc: 0.748626
[0.7440540710017827, 0.7486255232818275]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.727834
[200]	valid_0's auc: 0.730777
Early stopping, best iteration is:
[181]	valid_0's auc: 0.731173
[0.7440540710017827, 0.7486255232818275, 0.7311728919071874]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.738917
[200]	valid_0's auc: 0.738258
Early stopping, best iteration is:
[117]	valid_0's auc: 0.739563
[0

# ---分界线---

## 测试target encoder是否有效

In [9]:
import pandas as pd
DATA_PATH = '/kaggle/input/financial-risk/'
train_trans = pd.read_csv(DATA_PATH+'train_trans.csv')
test_trans = pd.read_csv(DATA_PATH+'test_a_trans.csv')
train_base = pd.read_csv(DATA_PATH+'train_base.csv')
train_label = pd.read_csv(DATA_PATH+'train_label.csv')
train_base = train_base.merge(train_label,on=['user'],how='left')
test_base= pd.read_csv(DATA_PATH+  'test_a_base.csv')
targer = pd.read_csv(DATA_PATH+'train_label.csv')

train_op = pd.read_csv(DATA_PATH+'train_op.csv')
test_op = pd.read_csv(DATA_PATH+'test_a_op.csv')

global orgin_features 
orgin_features = train_base.columns

In [10]:

train_base['city_balance_avg'] = train_base['city'].map(str) + '_' + train_base['balance_avg'].map(str)
train_base['city_level'] = train_base['city'].map(str) + '_' + train_base['level'].map(str)
train_base['city_product1_amount'] = train_base['city'].map(str) + '_' + train_base['product1_amount'].map(str)
train_base['city_balance1_avg'] = train_base['city'].map(str) + '_' + train_base['balance1_avg'].map(str)
train_base['city_balance2_avg'] = train_base['city'].map(str) + '_' + train_base['balance2_avg'].map(str)

test_base['city_balance_avg'] =  test_base['city'].map(str) + '_' + test_base['balance_avg'].map(str)
test_base['city_level'] =        test_base['city'].map(str) + '_' + test_base['level'].map(str)
test_base['city_product1_amount'] = test_base['city'].map(str) + '_' + test_base['product1_amount'].map(str)
test_base['city_balance1_avg'] = test_base['city'].map(str) + '_' + test_base['balance1_avg'].map(str)
test_base['city_balance2_avg'] = test_base['city'].map(str) + '_' + test_base['balance2_avg'].map(str)


### 自己定义的target encoder

In [11]:
def target_encoding_1(train,test, feature, k,noise_level=0.01,smoothing=20):
    
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)    

    for col in feature:
        train['target_'+col] = 0
        test['target_' + col] =0
        if col not in orgin_features:
            smoothing=50
            noise_level=0.7
        for trn_idx,val_idx in kf.split(train,train['label']):
            X = train.iloc[trn_idx][col]
            enc_tar = TargetEncoder(smoothing=smoothing).fit(X,train.iloc[trn_idx]['label'])
            train['target_'+col] += add_noise(enc_tar.transform(train[col])[col],noise_level) / k
            test['target_'+col] += add_noise(enc_tar.transform(test[col])[col],noise_level) / k
       
    return  train,test



def Catboost_encoding(train,test, feature, k,noise_level=0.01,sigma=2.0,a=1):
    
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)    

    for col in feature:
        train['catboost_'+col] = 0
        test['catboost_' + col] =0
        if col not in orgin_features:
            sigma=5.0
            a = 2
        for trn_idx,val_idx in kf.split(train,train['label']):
            X = train.iloc[trn_idx][col]
            enc_tar = CatBoostEncoder(sigma=sigma,a=a).fit(X,train.iloc[trn_idx]['label'])
            train['catboost_'+col] += add_noise(enc_tar.transform(train[col])[col],noise_level) / k
            test['catboost_'+col] += add_noise(enc_tar.transform(test[col])[col],noise_level) / k        
    return  train,test

In [12]:
# 评价编码函数
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)
f_cats = ['province','city',"city_level",'city_balance1_avg','city_balance2_avg',
                      "city_product1_amount",'city_balance_avg']

print("%20s   %20s | %20s | %20s  | %20s" % ("", "Raw Categories","Encoded Categories","Baseline Categories","CatBoost Categories"))
for f in f_cats:
    print("%-20s : " % f, end="")
    origin_scores = []
    baseline_scores = []
    target_scores=[]   
    catboost_scores=[]
    for trn_idx, val_idx in folds.split(train_base, train_base.label):
        
        val_f = train_base.iloc[trn_idx].groupby([f])['label'].transform('mean')
        val_f.name=f
        val_tgt = train_base.label.iloc[trn_idx]
        
        trn_f = train_base.iloc[trn_idx][f]
        trn_tgt = train_base.label.iloc[trn_idx]
        
        val_tf ,temp = target_encoding_1(train_base.iloc[trn_idx],test_base,[f],5)
        val_tf1 ,temp = kfold_stats_feature(train_base.iloc[trn_idx].reset_index(),test_base,[f],5)  
        val_tf2 ,temp = Catboost_encoding(train_base.iloc[trn_idx],test_base,[f],5)
        
        origin_scores.append(max(roc_auc_score(val_tgt, val_f), 1 - roc_auc_score(val_tgt, val_f)))
        target_scores.append(roc_auc_score(val_tgt, val_tf['target_'+f]))
        baseline_scores.append(roc_auc_score(val_tgt,val_tf1[f+'_label_kfold_mean']))
        catboost_scores.append(roc_auc_score(val_tgt, val_tf2['catboost_'+f]))
    val_tf1 ,temp = kfold_stats_feature(train_base,test_base,[f],5)    
    print(" %.6f + %.6f | %6f + %.6f  | %.6f + %.6f  | %.6f + %.6f" 
          % (np.mean( origin_scores), np.std( origin_scores), np.mean(target_scores), np.std(target_scores),
             np.mean(baseline_scores), np.std(baseline_scores),np.mean(catboost_scores), np.std(catboost_scores)))

                             Raw Categories |   Encoded Categories |  Baseline Categories  |  CatBoost Categories
province             :  0.571175 + 0.000598 | 0.571213 + 0.000621  | 0.566010 + 0.001213  | 0.571106 + 0.000575
city                 :  0.599885 + 0.000483 | 0.599572 + 0.000446  | 0.564816 + 0.001815  | 0.599797 + 0.000481
city_level           :  0.619688 + 0.000586 | 0.571063 + 0.003032  | 0.561827 + 0.002352  | 0.616616 + 0.000635
city_balance1_avg    :  0.734679 + 0.000766 | 0.648458 + 0.001810  | 0.572507 + 0.002086  | 0.704834 + 0.000692
city_balance2_avg    :  0.651628 + 0.000864 | 0.595657 + 0.002532  | 0.567518 + 0.002903  | 0.646448 + 0.000937
city_product1_amount :  0.646222 + 0.000623 | 0.594867 + 0.002254  | 0.591823 + 0.001756  | 0.642557 + 0.000612
city_balance_avg     :  0.761732 + 0.000644 | 0.674203 + 0.001903  | 0.547982 + 0.001843  | 0.737928 + 0.000764


In [13]:
#  自己写target encoder的评价方法,没有采用5折方法
def encode_val(train_base,test_base,cols):
    for col in cols:
    
        temp_train,temp_test=kfold_stats_feature(train_base,test_base,[col],5)
        temp=train_base.groupby([col])['label'].agg(cnt='count',mean='mean')
        temp=temp.reset_index()
        temp_train=temp_train.merge(temp,on=[col],how='left')
        
        
        print("BASELINE Encode score: {}".format(roc_auc_score(temp_train['label'],temp_train['{}_label_kfold_mean'.format(col)])))
        print("Myown Encode score:    {}".format(roc_auc_score(temp_train['label'],temp_train['Mean_encoded_{}'.format(col)])))
        print("Origin score:          {}".format(roc_auc_score(temp_train['label'],temp_train['mean'])))
        print()
cols = ['province','city',"city_level",'city_balance1_avg','city_balance2_avg',
                      "city_product1_amount",'city_balance_avg']
encode_val(train_base,test_base,cols)

BASELINE Encode score: 0.5667795997751115


KeyError: 'Mean_encoded_province'